####  Book Analysis
#### That Program go to a book website takes the relevant information for the further analysis of the books

Web site: [link]()

In [1]:
import requests
import pandas as pd
import pandasql as ps
from bs4 import BeautifulSoup

In [2]:

def crawl(page):
    url = "https://www.bookdepository.com/category/352/Science-Fiction/browse/viewmode/all?page=" + str(page)
    html = requests.get(url)
    soup = BeautifulSoup(html.text, "html.parser") 
    results = soup.find(class_='tab search')
    books = results.find_all(class_='book-item')
    
    return books

In [19]:
def books_info_to_table(page_info):
    book_table = []
    for book in page_info:
        try:
            title = book.find(class_='title').text.strip()
            author = book.find(class_='author').text.strip()
            isbn = book.find("meta", itemprop="isbn")["content"]
            discount_text = book.find(class_='price-save').text.strip() if book.find(class_='price-save') is not None else None
            discount = float(discount_text.split()[1].replace(",",".")) if discount_text is not None else 0
            price_text = book.find(class_='sale-price').text.strip() 
            price = float(price_text.split()[0].replace(",",".")) if price_text is not None else 0
            book_format = book.find(class_='format').text.strip()
            publish_date = book.find(class_='published').text.strip()
            full_start_rating  = book.find('div','rating-wrap').findAll('span', 'full-star')
            half_start  = book.find('div','rating-wrap').findAll('span', 'half-star')
            total_rating = len(full_start_rating) + len(half_start) * 0.5 
            book_row = [isbn, title, author, price, discount, book_format, publish_date, total_rating]
            book_table.append(book_row)
        except:
            print("error")
            continue
    return book_table

In [20]:
main_table = []
for x in range(1, 2):
    page = crawl(x)
    columns_name = ['isbn', 'title', 'author', 'price', 'discount', 'book_format', 'publish_date', 'total_rating']
    page_book_table = books_info_to_table(page)
    main_table += page_book_table

In [21]:
df_books = pd.DataFrame(main_table, columns=columns_name)

In [22]:
df_books.head()

,isbn,title,author,price,discount,book_format,publish_date,total_rating
0,9780141036144,1984,George Orwell,11.95,0.00,Paperback,01 Oct 2008,4.0
1,9780141036137,Animal Farm,George Orwell,10.56,0.00,Paperback,01 Oct 2008,4.0
2,9781408725764,The Love Hypothesis,Ali Hazelwood,9.13,3.37,Paperback,21 Oct 2021,4.5
3,9780340960196,Dune,Frank Herbert,10.32,3.18,Paperback,23 Jul 2015,4.0
4,9780099560432,Ready Player One,Ernest Cline,8.33,4.17,Paperback,05 Apr 2012,4.0


**Find the 10 books with a bigger discount that at least have a rating of 4 stars. The prices should be less than 12€ as well**

In [34]:
ps.sqldf("""select  
                title, 
                author, 
                price, 
                discount, 
                total_rating 
            from df_books 
            where total_rating >= 4.0 and price <= 12.0
            order by discount  desc   
            limit 10  """)

,title,author,price,discount,total_rating
0,Do Androids Dream Of Electric Sheep?: The insp...,Philip K. Dick,6.50,5.50,4.0
1,Ready Player One,Ernest Cline,8.33,4.17,4.0
2,The Love Hypothesis,Ali Hazelwood,9.13,3.37,4.5
3,Dune,Frank Herbert,10.32,3.18,4.0
4,Dune Messiah,Frank Herbert,9.99,3.01,4.0
5,Cloud Atlas,David Mitchell,10.24,2.76,4.0
6,"The Long Way to a Small, Angry Planet",Becky Chambers,9.57,2.43,4.0
7,Queen of Shadows,Sarah J. Maas,10.33,2.17,4.5
8,Mere Christianity,C. S. Lewis,9.99,2.01,4.5
9,Never Let Me Go,Kazuo Ishiguro,10.46,1.54,4.0


**--End--**